# PyTorch Tensor Fundamentals: Comprehensive Deep Learning Foundation Analysis

**PyTorch Mastery Hub - Fundamentals Module**

**Authors:** PyTorch Mastery Hub Development Team  
**Institution:** PyTorch Mastery Hub  
**Course:** Deep Learning Fundamentals with PyTorch  

## Overview

This notebook provides comprehensive analysis and exploration of PyTorch tensors, the fundamental building blocks of deep learning. We focus on understanding tensor operations, memory management, performance optimization, and practical applications to build a solid foundation for advanced deep learning concepts.

## Key Objectives
1. Master tensor creation, manipulation, and operation techniques
2. Understand memory management and performance optimization strategies
3. Explore GPU acceleration and device management best practices
4. Analyze broadcasting, reshaping, and advanced indexing capabilities
5. Implement practical machine learning applications using tensor operations
6. Establish performance benchmarking and optimization methodologies

## 1. Setup and Environment Configuration

In [ ]:
# Essential imports for comprehensive tensor analysis
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import time
import sys
import warnings
from pathlib import Path
from collections import defaultdict
import json
import psutil
import gc
import math  

warnings.filterwarnings('ignore')

# Add custom modules to path
sys.path.append(str(Path.cwd().parent.parent / "src"))

# Import custom utilities
try:
    from fundamentals import TensorOperations, print_system_info
    from utils.device_utils import get_device
    from utils.memory_utils import MemoryTracker
except ImportError:
    print("⚠️ Custom utilities not found, using fallback implementations")
    
    def print_system_info():
        print(f"PyTorch version: {torch.__version__}")
        print(f"CUDA available: {torch.cuda.is_available()}")
        if torch.cuda.is_available():
            print(f"CUDA version: {torch.version.cuda}")
            print(f"GPU count: {torch.cuda.device_count()}")
    
    def get_device():
        if torch.cuda.is_available():
            return torch.device("cuda")
        elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
            return torch.device("mps")
        else:
            return torch.device("cpu")

# Enhanced plotting configuration
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Create results directory
results_dir = Path('../results/notebooks/tensor_fundamentals')
results_dir.mkdir(parents=True, exist_ok=True)

# Initialize analysis tracking
analysis_results = {
    'system_info': {},
    'tensor_creation_analysis': {},
    'performance_benchmarks': {},
    'memory_usage_analysis': {},
    'operation_comparisons': {},
    'device_performance': {}
}

print("🔥 PyTorch Mastery Hub - Tensor Fundamentals Analysis")
print("=" * 60)
print(f"📁 Results directory: {results_dir}")
print(f"🎯 Comprehensive tensor analysis initialized")
print()

# System information analysis
print("🖥️ System Environment Analysis")
print("-" * 35)
print_system_info()

# Device detection and analysis
device = get_device()
print(f"\n🎯 Primary compute device: {device}")

# Store system information
analysis_results['system_info'] = {
    'pytorch_version': torch.__version__,
    'cuda_available': torch.cuda.is_available(),
    'mps_available': hasattr(torch.backends, 'mps') and torch.backends.mps.is_available(),
    'primary_device': str(device),
    'cpu_count': psutil.cpu_count(),
    'total_memory_gb': psutil.virtual_memory().total / (1024**3)
}

if torch.cuda.is_available():
    analysis_results['system_info'].update({
        'cuda_version': torch.version.cuda,
        'gpu_count': torch.cuda.device_count(),
        'gpu_name': torch.cuda.get_device_name(),
        'gpu_memory_gb': torch.cuda.get_device_properties(0).total_memory / (1024**3)
    })

print("✅ Environment setup complete!")

# Add error handling and robustness functions here
def safe_device_operation(operation_func, *args, **kwargs):
    """Safely execute operations with device fallback."""
    try:
        return operation_func(*args, **kwargs)
    except RuntimeError as e:
        if "CUDA" in str(e) or "MPS" in str(e):
            print(f"⚠️ Device operation failed, falling back to CPU: {e}")
            # Move tensors to CPU and retry
            cpu_args = [arg.cpu() if hasattr(arg, 'cpu') else arg for arg in args]
            cpu_kwargs = {k: v.cpu() if hasattr(v, 'cpu') else v for k, v in kwargs.items()}
            return operation_func(*cpu_args, **cpu_kwargs)
        else:
            raise e

def safe_memory_operation(func, *args, **kwargs):
    """Execute with memory management."""
    try:
        return func(*args, **kwargs)
    except RuntimeError as e:
        if "out of memory" in str(e).lower():
            print(f"⚠️ Out of memory, attempting cleanup...")
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            gc.collect()
            return None
        else:
            raise e

print("🛡️ Error handling functions loaded")

# Performance optimization functions
def get_adaptive_tensor_sizes(device):
    """Get appropriate tensor sizes based on device capability."""
    if device.type == 'cuda':
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
        if gpu_memory > 8:
            return [(500, 500), (1000, 1000), (2000, 2000)]
        else:
            return [(300, 300), (600, 600), (1000, 1000)]
    elif device.type == 'mps':
        return [(300, 300), (600, 600), (1000, 1000)]
    else:
        return [(200, 200), (500, 500), (800, 800)]

def get_adaptive_iterations(operation_type, device):
    """Get appropriate number of iterations based on operation and device."""
    base_iterations = {
        'fast_ops': 50,
        'medium_ops': 20,
        'slow_ops': 5
    }
    
    if device.type == 'cpu':
        return {k: max(1, v // 2) for k, v in base_iterations.items()}
    else:
        return base_iterations

# Get adaptive settings for this device
adaptive_sizes = get_adaptive_tensor_sizes(device)
adaptive_iterations = get_adaptive_iterations('medium_ops', device)

print(f"⚡ Performance optimizations loaded for {device}")
print(f"   Adaptive tensor sizes: {adaptive_sizes}")
print(f"   Adaptive iterations: {adaptive_iterations}")

## 2. Comprehensive Tensor Creation Analysis

### 2.1 Tensor Creation Methods and Performance

In [ ]:
def analyze_tensor_creation_methods():
    """Comprehensive analysis of tensor creation methods with performance metrics."""
    
    print("🏗️ Tensor Creation Methods Analysis")
    print("=" * 40)
    
    creation_methods = {}
    performance_metrics = {}
    memory_usage = {}
    
    # Define test parameters
    test_sizes = [(100, 100), (500, 500), (1000, 1000)]
    #test_sizes = get_adaptive_tensor_sizes(device)
    
    for size in test_sizes:
        print(f"\n📊 Analyzing size {size[0]}x{size[1]}...")
        
        size_results = {}
        size_performance = {}
        size_memory = {}
        
        # Method 1: Zeros tensor
        start_time = time.time()
        zeros_tensor = torch.zeros(size, device=device)
        zeros_time = time.time() - start_time
        zeros_memory = zeros_tensor.numel() * zeros_tensor.element_size()
        
        size_results['zeros'] = {
            'tensor': zeros_tensor,
            'mean': float(zeros_tensor.mean()),
            'std': float(zeros_tensor.std()),
            'dtype': str(zeros_tensor.dtype),
            'shape': zeros_tensor.shape
        }
        size_performance['zeros'] = zeros_time
        size_memory['zeros'] = zeros_memory
        
        # Method 2: Ones tensor
        start_time = time.time()
        ones_tensor = torch.ones(size, device=device)
        ones_time = time.time() - start_time
        ones_memory = ones_tensor.numel() * ones_tensor.element_size()
        
        size_results['ones'] = {
            'tensor': ones_tensor,
            'mean': float(ones_tensor.mean()),
            'std': float(ones_tensor.std()),
            'dtype': str(ones_tensor.dtype),
            'shape': ones_tensor.shape
        }
        size_performance['ones'] = ones_time
        size_memory['ones'] = ones_memory
        
        # Method 3: Random normal
        start_time = time.time()
        randn_tensor = torch.randn(size, device=device)
        randn_time = time.time() - start_time
        randn_memory = randn_tensor.numel() * randn_tensor.element_size()
        
        size_results['randn'] = {
            'tensor': randn_tensor,
            'mean': float(randn_tensor.mean()),
            'std': float(randn_tensor.std()),
            'dtype': str(randn_tensor.dtype),
            'shape': randn_tensor.shape
        }
        size_performance['randn'] = randn_time
        size_memory['randn'] = randn_memory
        
        # Method 4: Random uniform
        start_time = time.time()
        rand_tensor = torch.rand(size, device=device)
        rand_time = time.time() - start_time
        rand_memory = rand_tensor.numel() * rand_tensor.element_size()
        
        size_results['rand'] = {
            'tensor': rand_tensor,
            'mean': float(rand_tensor.mean()),
            'std': float(rand_tensor.std()),
            'dtype': str(rand_tensor.dtype),
            'shape': rand_tensor.shape
        }
        size_performance['rand'] = rand_time
        size_memory['rand'] = rand_memory
        
        # Method 5: Xavier uniform initialization
        start_time = time.time()
        xavier_tensor = torch.empty(size, device=device)
        torch.nn.init.xavier_uniform_(xavier_tensor)
        xavier_time = time.time() - start_time
        xavier_memory = xavier_tensor.numel() * xavier_tensor.element_size()
        
        size_results['xavier'] = {
            'tensor': xavier_tensor,
            'mean': float(xavier_tensor.mean()),
            'std': float(xavier_tensor.std()),
            'dtype': str(xavier_tensor.dtype),
            'shape': xavier_tensor.shape
        }
        size_performance['xavier'] = xavier_time
        size_memory['xavier'] = xavier_memory
        
        # Method 6: Kaiming normal initialization
        start_time = time.time()
        kaiming_tensor = torch.empty(size, device=device)
        torch.nn.init.kaiming_normal_(kaiming_tensor)
        kaiming_time = time.time() - start_time
        kaiming_memory = kaiming_tensor.numel() * kaiming_tensor.element_size()
        
        size_results['kaiming'] = {
            'tensor': kaiming_tensor,
            'mean': float(kaiming_tensor.mean()),
            'std': float(kaiming_tensor.std()),
            'dtype': str(kaiming_tensor.dtype),
            'shape': kaiming_tensor.shape
        }
        size_performance['kaiming'] = kaiming_time
        size_memory['kaiming'] = kaiming_memory
        
        creation_methods[f"{size[0]}x{size[1]}"] = size_results
        performance_metrics[f"{size[0]}x{size[1]}"] = size_performance
        memory_usage[f"{size[0]}x{size[1]}"] = size_memory
        
        # Print statistics for this size
        for method, results in size_results.items():
            print(f"  {method.capitalize():10} - Mean: {results['mean']:8.4f}, "
                  f"Std: {results['std']:8.4f}, Time: {size_performance[method]:.6f}s")
    
    return creation_methods, performance_metrics, memory_usage

# Run tensor creation analysis
creation_data, creation_performance, creation_memory = analyze_tensor_creation_methods()

# Store results
analysis_results['tensor_creation_analysis'] = {
    'creation_methods': creation_data,
    'performance_metrics': creation_performance,
    'memory_usage': creation_memory
}

print(f"\n💾 Tensor creation analysis completed")

### 2.2 Tensor Creation Visualization and Comparison

In [ ]:
def visualize_tensor_creation_analysis(creation_data, performance_data, memory_data):
    """Create comprehensive visualizations for tensor creation analysis."""
    
    print("\n🎨 Creating Tensor Creation Visualizations")
    print("-" * 45)
    
    # Create comprehensive visualization dashboard
    fig = plt.figure(figsize=(20, 16))
    gs = fig.add_gridspec(4, 4, hspace=0.3, wspace=0.3)
    
    # 1. Tensor distribution visualization (2D heatmaps)
    size_key = "1000x1000"
    if size_key in creation_data:
        sample_data = creation_data[size_key]
        
        # Plot different initialization methods
        methods_to_plot = ['zeros', 'ones', 'randn', 'rand', 'xavier', 'kaiming']
        
        for i, method in enumerate(methods_to_plot):
            if method in sample_data:
                ax = fig.add_subplot(gs[0, i % 3])
                tensor_np = sample_data[method]['tensor'].cpu().numpy()
                
                # Sample for visualization (avoid memory issues)
                if tensor_np.shape[0] > 100:
                    tensor_np = tensor_np[:100, :100]
                
                im = ax.imshow(tensor_np, cmap='viridis', aspect='auto')
                ax.set_title(f'{method.capitalize()}\nMean: {sample_data[method]["mean"]:.3f}')
                plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
                
                if i >= 3:  # Second row
                    ax = fig.add_subplot(gs[1, (i-3) % 3])
                    im = ax.imshow(tensor_np, cmap='viridis', aspect='auto')
                    ax.set_title(f'{method.capitalize()}\nStd: {sample_data[method]["std"]:.3f}')
                    plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    
    # 2. Performance comparison across sizes
    ax_perf = fig.add_subplot(gs[2, :2])
    
    sizes = list(performance_data.keys())
    methods = list(performance_data[sizes[0]].keys())
    
    x = np.arange(len(sizes))
    width = 0.12
    
    for i, method in enumerate(methods):
        times = [performance_data[size][method] for size in sizes]
        bars = ax_perf.bar(x + i * width, times, width, label=method.capitalize(), alpha=0.8)
        
        # Add value labels on bars
        for bar, time_val in zip(bars, times):
            height = bar.get_height()
            ax_perf.text(bar.get_x() + bar.get_width()/2., height + max(times)*0.01,
                        f'{time_val:.4f}', ha='center', va='bottom', fontsize=8)
    
    ax_perf.set_xlabel('Tensor Size')
    ax_perf.set_ylabel('Creation Time (seconds)')
    ax_perf.set_title('Tensor Creation Performance Comparison')
    ax_perf.set_xticks(x + width * (len(methods) - 1) / 2)
    ax_perf.set_xticklabels(sizes)
    ax_perf.legend()
    ax_perf.grid(True, alpha=0.3)
    
    # 3. Memory usage analysis
    ax_mem = fig.add_subplot(gs[2, 2:])
    
    # Memory usage is the same for all methods (same size), so show one method
    if sizes and methods:
        memory_values = [memory_data[size][methods[0]] / (1024**2) for size in sizes]  # Convert to MB
        
        bars = ax_mem.bar(sizes, memory_values, alpha=0.7, color='lightblue')
        
        for bar, mem_val in zip(bars, memory_values):
            height = bar.get_height()
            ax_mem.text(bar.get_x() + bar.get_width()/2., height + max(memory_values)*0.01,
                       f'{mem_val:.1f}MB', ha='center', va='bottom', fontsize=10)
        
        ax_mem.set_xlabel('Tensor Size')
        ax_mem.set_ylabel('Memory Usage (MB)')
        ax_mem.set_title('Memory Usage by Tensor Size')
        ax_mem.grid(True, alpha=0.3)
    
    # 4. Statistical distribution analysis
    ax_stats = fig.add_subplot(gs[3, :2])
    
    if size_key in creation_data:
        sample_data = creation_data[size_key]
        
        means = [sample_data[method]['mean'] for method in methods if method in sample_data]
        stds = [sample_data[method]['std'] for method in methods if method in sample_data]
        method_names = [method.capitalize() for method in methods if method in sample_data]
        
        x = np.arange(len(method_names))
        width = 0.35
        
        bars1 = ax_stats.bar(x - width/2, means, width, label='Mean', alpha=0.8, color='skyblue')
        bars2 = ax_stats.bar(x + width/2, stds, width, label='Standard Deviation', alpha=0.8, color='lightcoral')
        
        # Add value labels
        for bars in [bars1, bars2]:
            for bar in bars:
                height = bar.get_height()
                ax_stats.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                             f'{height:.3f}', ha='center', va='bottom', fontsize=9)
        
        ax_stats.set_xlabel('Initialization Method')
        ax_stats.set_ylabel('Value')
        ax_stats.set_title('Statistical Properties of Different Initializations')
        ax_stats.set_xticks(x)
        ax_stats.set_xticklabels(method_names, rotation=45, ha='right')
        ax_stats.legend()
        ax_stats.grid(True, alpha=0.3)
    
    # 5. Comprehensive summary
    ax_summary = fig.add_subplot(gs[3, 2:])
    
    summary_text = f"""
🔥 TENSOR CREATION ANALYSIS SUMMARY

📊 Methods Analyzed: {len(methods)}
📏 Size Variations: {len(sizes)}
🖥️ Compute Device: {device}

⚡ Performance Insights:
• Fastest Method: {min(performance_data[sizes[-1]], key=performance_data[sizes[-1]].get).capitalize()}
• Memory Efficient: All methods use equal memory for same size
• Statistical Properties: Xavier/Kaiming optimized for deep learning

🎯 Recommendations:
• Use zeros/ones for simple initialization
• Use randn for general random initialization  
• Use Xavier/Kaiming for neural network weights
• Consider memory usage for large tensors
• GPU acceleration provides significant speedup

📈 Key Findings:
• Creation time scales with tensor size
• Initialization method affects statistical properties
• Memory usage is consistent across methods
• Device selection impacts performance significantly
    """
    
    ax_summary.text(0.05, 0.95, summary_text, transform=ax_summary.transAxes, fontsize=11,
                   verticalalignment='top', horizontalalignment='left',
                   bbox=dict(boxstyle='round', facecolor='lightcyan', alpha=0.8))
    ax_summary.set_title('Analysis Summary & Recommendations', fontweight='bold')
    ax_summary.axis('off')
    
    plt.suptitle('Comprehensive Tensor Creation Analysis Dashboard', fontsize=18, fontweight='bold')
    
    # Save visualization
    plt.savefig(results_dir / 'tensor_creation_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    return fig

# Create visualization
creation_viz_fig = visualize_tensor_creation_analysis(creation_data, creation_performance, creation_memory)
print("📊 Tensor creation visualizations completed")

## 3. Comprehensive Tensor Operations Analysis

### 3.1 Basic and Advanced Operations Benchmarking

In [ ]:
def analyze_tensor_operations():
    """Comprehensive analysis of tensor operations with performance benchmarking."""
    
    print("\n🔢 Tensor Operations Performance Analysis")
    print("=" * 45)
    
    # Test parameters
    test_sizes = [(500, 500), (1000, 1000), (2000, 2000)]
    operations_results = {}
    
    for size in test_sizes:
        print(f"\n📊 Analyzing operations for size {size[0]}x{size[1]}...")
        
        # Create test tensors
        a = torch.randn(size, device=device)
        b = torch.randn(size, device=device)
        
        size_results = {}
        
        # Element-wise operations
        operations = {
            'addition': lambda x, y: x + y,
            'subtraction': lambda x, y: x - y,
            'multiplication': lambda x, y: x * y,
            'division': lambda x, y: x / y,
            'power': lambda x, y: x ** 2,
            'sqrt': lambda x, y: torch.sqrt(torch.abs(x)),
            'exp': lambda x, y: torch.exp(x / 10),  # Scaled to avoid overflow
            'log': lambda x, y: torch.log(torch.abs(x) + 1e-8),
            'sin': lambda x, y: torch.sin(x),
            'cos': lambda x, y: torch.cos(x)
        }
        
        for op_name, op_func in operations.items():
            # Benchmark operation
            num_runs = 10
            times = []
            
            for _ in range(num_runs):
                if device.type == 'cuda':
                    torch.cuda.synchronize()
                
                start_time = time.time()
                result = op_func(a, b)
                
                if device.type == 'cuda':
                    torch.cuda.synchronize()
                
                end_time = time.time()
                times.append(end_time - start_time)
            
            avg_time = np.mean(times)
            std_time = np.std(times)
            
            # Calculate result statistics
            result_stats = {
                'mean': float(result.mean()),
                'std': float(result.std()),
                'min': float(result.min()),
                'max': float(result.max()),
                'shape': result.shape
            }
            
            size_results[op_name] = {
                'avg_time': avg_time,
                'std_time': std_time,
                'stats': result_stats,
                'throughput': result.numel() / avg_time  # Elements per second
            }
            
            print(f"  {op_name:12} - Time: {avg_time:.6f}±{std_time:.6f}s, "
                  f"Throughput: {size_results[op_name]['throughput']:.2e} elem/s")
        
        operations_results[f"{size[0]}x{size[1]}"] = size_results
    
    return operations_results

# Run operations analysis
operations_data = analyze_tensor_operations()

# Store results
analysis_results['operation_comparisons'] = operations_data

print(f"\n💾 Tensor operations analysis completed")

### 3.2 Matrix Operations and Linear Algebra Analysis

In [ ]:
def analyze_matrix_operations():
    """Comprehensive analysis of matrix operations for deep learning."""
    
    print("\n🧮 Matrix Operations Analysis")
    print("=" * 35)
    
    matrix_results = {}
    
    # Test different matrix sizes
    test_configs = [
        {'name': 'Small', 'size': (128, 128)},
        {'name': 'Medium', 'size': (512, 512)},
        {'name': 'Large', 'size': (1024, 1024)}
    ]
    
    for config in test_configs:
        size = config['size']
        name = config['name']
        
        print(f"\n📊 Analyzing {name} matrices ({size[0]}x{size[1]})...")
        
        # Create test matrices
        A = torch.randn(size, device=device)
        B = torch.randn(size, device=device)
        
        config_results = {}
        
        # Matrix multiplication
        start_time = time.time()
        matmul_result = torch.matmul(A, B)
        if device.type == 'cuda':
            torch.cuda.synchronize()
        matmul_time = time.time() - start_time
        
        config_results['matmul'] = {
            'time': matmul_time,
            'shape': matmul_result.shape,
            'flops': 2 * size[0] * size[1] * size[1],  # 2*n^3 for n×n matrix multiplication
            'gflops': (2 * size[0] * size[1] * size[1]) / (matmul_time * 1e9)
        }
        
        # Transpose
        start_time = time.time()
        transpose_result = A.t()
        transpose_time = time.time() - start_time
        
        config_results['transpose'] = {
            'time': transpose_time,
            'shape': transpose_result.shape
        }
        
        # Determinant (for square matrices)
        start_time = time.time()
        det_result = torch.det(A)
        det_time = time.time() - start_time
        
        config_results['determinant'] = {
            'time': det_time,
            'value': float(det_result)
        }
        
        # Eigenvalues (expensive operation)
        if size[0] <= 512:  # Only for smaller matrices
            start_time = time.time()
            eigenvals = torch.linalg.eigvals(A)
            eigen_time = time.time() - start_time
            
            config_results['eigenvalues'] = {
                'time': eigen_time,
                'count': len(eigenvals),
                'real_part_mean': float(eigenvals.real.mean()),
                'imag_part_mean': float(eigenvals.imag.mean())
            }
        
        # SVD (Singular Value Decomposition)
        if size[0] <= 512:  # Only for smaller matrices
            start_time = time.time()
            U, S, Vh = torch.linalg.svd(A)
            svd_time = time.time() - start_time
            
            config_results['svd'] = {
                'time': svd_time,
                'singular_values_shape': S.shape,
                'condition_number': float(S.max() / S.min())
            }
        
        matrix_results[name] = config_results
        
        print(f"  Matrix Multiply: {matmul_time:.6f}s ({config_results['matmul']['gflops']:.2f} GFLOPS)")
        print(f"  Transpose:       {transpose_time:.6f}s")
        print(f"  Determinant:     {det_time:.6f}s (det = {det_result:.4f})")
        
        if 'eigenvalues' in config_results:
            print(f"  Eigenvalues:     {config_results['eigenvalues']['time']:.6f}s")
        if 'svd' in config_results:
            print(f"  SVD:             {config_results['svd']['time']:.6f}s")
    
    return matrix_results

# Run matrix operations analysis
matrix_data = analyze_matrix_operations()

# Store results
analysis_results['matrix_operations'] = matrix_data

print(f"\n💾 Matrix operations analysis completed")

## 4. Broadcasting and Shape Manipulation Analysis

### 4.1 Broadcasting Capabilities and Performance

In [ ]:
def analyze_broadcasting_operations():
    """Comprehensive analysis of broadcasting operations and performance."""
    
    print("\n📡 Broadcasting Operations Analysis")
    print("=" * 40)
    
    broadcasting_results = {}
    
    # Test different broadcasting scenarios
    broadcast_tests = [
        {
            'name': 'Scalar_Tensor',
            'desc': 'Scalar + Tensor',
            'setup': lambda: (torch.tensor(2.0, device=device), torch.randn(1000, 1000, device=device)),
            'operation': lambda a, b: a + b
        },
        {
            'name': 'Vector_Matrix',
            'desc': 'Vector + Matrix',
            'setup': lambda: (torch.randn(1000, device=device), torch.randn(1000, 1000, device=device)),
            'operation': lambda a, b: a + b
        },
        {
            'name': 'Different_Dims',
            'desc': 'Different Dimensions',
            'setup': lambda: (torch.randn(1, 100, 1000, device=device), torch.randn(50, 1, 1000, device=device)),
            'operation': lambda a, b: a * b
        },
        {
            'name': 'Matrix_Batch',
            'desc': 'Matrix + Batch',
            'setup': lambda: (torch.randn(32, 1, 100, device=device), torch.randn(1, 50, 1, device=device)),
            'operation': lambda a, b: a + b
        }
    ]
    
    for test in broadcast_tests:
        print(f"\n📊 Testing {test['desc']}...")
        
        # Setup tensors
        a, b = test['setup']()
        
        print(f"  Tensor A shape: {a.shape}")
        print(f"  Tensor B shape: {b.shape}")
        
        # Benchmark broadcasting operation
        num_runs = 10
        times = []
        
        for _ in range(num_runs):
            if device.type == 'cuda':
                torch.cuda.synchronize()
            
            start_time = time.time()
            result = test['operation'](a, b)
            
            if device.type == 'cuda':
                torch.cuda.synchronize()
            
            end_time = time.time()
            times.append(end_time - start_time)
        
        avg_time = np.mean(times)
        std_time = np.std(times)
        
        broadcasting_results[test['name']] = {
            'description': test['desc'],
            'input_shapes': {'a': a.shape, 'b': b.shape},
            'output_shape': result.shape,
            'avg_time': avg_time,
            'std_time': std_time,
            'memory_efficiency': (a.numel() + b.numel()) / result.numel(),
            'throughput': result.numel() / avg_time
        }
        
        print(f"  Result shape: {result.shape}")
        print(f"  Time: {avg_time:.6f}±{std_time:.6f}s")
        print(f"  Memory efficiency: {broadcasting_results[test['name']]['memory_efficiency']:.2f}")
    
    return broadcasting_results

# Run broadcasting analysis
broadcasting_data = analyze_broadcasting_operations()

# Store results
analysis_results['broadcasting_analysis'] = broadcasting_data

print(f"\n💾 Broadcasting analysis completed")

### 4.2 Shape Manipulation and Reshaping Analysis

In [ ]:
def analyze_shape_operations():
    """Comprehensive analysis of shape manipulation operations."""
    
    print("\n🔄 Shape Manipulation Analysis")
    print("=" * 35)
    
    shape_results = {}
    
    # Create test tensor
    original_tensor = torch.randn(8, 16, 32, 64, device=device)
    print(f"Original tensor shape: {original_tensor.shape}")
    print(f"Total elements: {original_tensor.numel()}")
    
    # Define shape operations
    shape_operations = {
        'flatten': {
            'operation': lambda x: x.flatten(),
            'description': 'Flatten all dimensions'
        },
        'view_2d': {
            'operation': lambda x: x.view(-1, 64),
            'description': 'Reshape to 2D (infer first dim)'
        },
        'reshape_4d': {
            'operation': lambda x: x.reshape(4, 32, 32, 64),
            'description': 'Reshape to different 4D'
        },
        'permute': {
            'operation': lambda x: x.permute(3, 2, 1, 0),
            'description': 'Permute dimensions'
        },
        'transpose': {
            'operation': lambda x: x.transpose(1, 2),
            'description': 'Transpose two dimensions'
        },
        'unsqueeze': {
            'operation': lambda x: x.unsqueeze(0),
            'description': 'Add batch dimension'
        },
        'squeeze': {
            'operation': lambda x: x.squeeze(),
            'description': 'Remove singleton dimensions'
        }
    }
    
    for op_name, op_info in shape_operations.items():
        print(f"\n📊 Testing {op_info['description']}...")
        
        try:
            # Benchmark operation
            num_runs = 100  # More runs for fast operations
            times = []
            
            for _ in range(num_runs):
                if device.type == 'cuda':
                    torch.cuda.synchronize()
                
                start_time = time.time()
                result = op_info['operation'](original_tensor)
                
                if device.type == 'cuda':
                    torch.cuda.synchronize()
                
                end_time = time.time()
                times.append(end_time - start_time)
            
            avg_time = np.mean(times)
            std_time = np.std(times)
            
            # Check if operation creates a view or copy
            shares_memory = result.data_ptr() == original_tensor.data_ptr()
            
            shape_results[op_name] = {
                'description': op_info['description'],
                'input_shape': original_tensor.shape,
                'output_shape': result.shape,
                'avg_time': avg_time,
                'std_time': std_time,
                'shares_memory': shares_memory,
                'is_contiguous': result.is_contiguous(),
                'elements_preserved': result.numel() == original_tensor.numel()
            }
            
            print(f"  Result shape: {result.shape}")
            print(f"  Time: {avg_time:.8f}±{std_time:.8f}s")
            print(f"  Shares memory: {shares_memory}")
            print(f"  Contiguous: {result.is_contiguous()}")
            
        except Exception as e:
            print(f"  Error: {e}")
            shape_results[op_name] = {
                'description': op_info['description'],
                'error': str(e)
            }
    
    return shape_results

# Run shape operations analysis
shape_data = analyze_shape_operations()

# Store results
analysis_results['shape_operations'] = shape_data

print(f"\n💾 Shape operations analysis completed")

## 5. Device Performance and Memory Analysis

### 5.1 CPU vs GPU Performance Comparison

In [ ]:
def analyze_device_performance():
    """Comprehensive device performance analysis comparing CPU, GPU, and MPS."""
    
    print("\n🚀 Device Performance Analysis")
    print("=" * 35)
    
    device_results = {}
    available_devices = ['cpu']
    
    # Check available devices
    if torch.cuda.is_available():
        available_devices.append('cuda')
    if hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
        available_devices.append('mps')
    
    print(f"Available devices: {available_devices}")
    
    # Test configurations
    test_configs = [
        {'name': 'Small Matrix Multiply', 'size': (512, 512), 'operation': 'matmul'},
        {'name': 'Large Matrix Multiply', 'size': (2048, 2048), 'operation': 'matmul'},
        {'name': 'Element-wise Operations', 'size': (1000, 1000), 'operation': 'elementwise'},
        {'name': 'Reduction Operations', 'size': (1000, 1000, 100), 'operation': 'reduction'}
    ]
    
    for config in test_configs:
        print(f"\n📊 Testing {config['name']}...")
        config_results = {}
        
        for device_name in available_devices:
            test_device = torch.device(device_name)
            print(f"  Testing on {device_name.upper()}...")
            
            try:
                if config['operation'] == 'matmul':
                    # Matrix multiplication test
                    a = torch.randn(config['size'], device=test_device)
                    b = torch.randn(config['size'], device=test_device)
                    
                    # Warm-up
                    for _ in range(3):
                        _ = torch.matmul(a, b)
                    
                    # Benchmark
                    times = []
                    num_runs = 10
                    
                    for _ in range(num_runs):
                        if test_device.type == 'cuda':
                            torch.cuda.synchronize()
                        
                        start_time = time.time()
                        result = torch.matmul(a, b)
                        
                        if test_device.type == 'cuda':
                            torch.cuda.synchronize()
                        
                        end_time = time.time()
                        times.append(end_time - start_time)
                    
                    flops = 2 * config['size'][0] * config['size'][1] * config['size'][1]
                    
                elif config['operation'] == 'elementwise':
                    # Element-wise operations test
                    a = torch.randn(config['size'], device=test_device)
                    b = torch.randn(config['size'], device=test_device)
                    
                    times = []
                    num_runs = 20
                    
                    for _ in range(num_runs):
                        if test_device.type == 'cuda':
                            torch.cuda.synchronize()
                        
                        start_time = time.time()
                        result = a + b * torch.sin(a) - torch.cos(b)
                        
                        if test_device.type == 'cuda':
                            torch.cuda.synchronize()
                        
                        end_time = time.time()
                        times.append(end_time - start_time)
                    
                    flops = a.numel() * 5  # Approximate FLOPS for the operations
                
                elif config['operation'] == 'reduction':
                    # Reduction operations test
                    a = torch.randn(config['size'], device=test_device)
                    
                    times = []
                    num_runs = 20
                    
                    for _ in range(num_runs):
                        if test_device.type == 'cuda':
                            torch.cuda.synchronize()
                        
                        start_time = time.time()
                        result = a.sum(dim=2).mean(dim=0).std()
                        
                        if test_device.type == 'cuda':
                            torch.cuda.synchronize()
                        
                        end_time = time.time()
                        times.append(end_time - start_time)
                    
                    flops = a.numel() * 3  # Approximate FLOPS
                
                avg_time = np.mean(times)
                std_time = np.std(times)
                
                config_results[device_name] = {
                    'avg_time': avg_time,
                    'std_time': std_time,
                    'gflops': flops / (avg_time * 1e9),
                    'throughput': a.numel() / avg_time,
                    'memory_gb': a.numel() * a.element_size() / (1024**3)
                }
                
                print(f"    Time: {avg_time:.6f}±{std_time:.6f}s")
                print(f"    GFLOPS: {config_results[device_name]['gflops']:.2f}")
                
            except Exception as e:
                print(f"    Error on {device_name}: {e}")
                config_results[device_name] = {'error': str(e)}
        
        device_results[config['name']] = config_results
    
    return device_results

# Run device performance analysis
device_performance_data = analyze_device_performance()

# Store results
analysis_results['device_performance'] = device_performance_data

print(f"\n💾 Device performance analysis completed")

### 5.2 Memory Management and Optimization Analysis

In [ ]:
def analyze_memory_management():
    """Comprehensive memory management analysis."""
    
    print("\n💾 Memory Management Analysis")
    print("=" * 35)
    
    memory_results = {}
    
    def get_memory_info():
        """Get current memory information."""
        if torch.cuda.is_available():
            return {
                'allocated_mb': torch.cuda.memory_allocated() / (1024**2),
                'reserved_mb': torch.cuda.memory_reserved() / (1024**2),
                'max_allocated_mb': torch.cuda.max_memory_allocated() / (1024**2)
            }
        else:
            # For CPU, use process memory
            process = psutil.Process()
            return {
                'allocated_mb': process.memory_info().rss / (1024**2),
                'reserved_mb': process.memory_info().vms / (1024**2),
                'max_allocated_mb': process.memory_info().rss / (1024**2)
            }
    
    # Initial memory state
    initial_memory = get_memory_info()
    print(f"Initial memory state: {initial_memory}")
    
    # Test 1: Memory allocation patterns
    print(f"\n📊 Testing memory allocation patterns...")
    
    allocation_test = {}
    tensor_sizes = [100, 500, 1000, 2000]
    
    for size in tensor_sizes:
        print(f"  Creating {size}x{size} tensor...")
        
        before_memory = get_memory_info()
        tensor = torch.randn(size, size, device=device)
        after_memory = get_memory_info()
        
        memory_used = after_memory['allocated_mb'] - before_memory['allocated_mb']
        expected_memory = (size * size * 4) / (1024**2)  # 4 bytes per float32
        
        allocation_test[f"{size}x{size}"] = {
            'actual_memory_mb': memory_used,
            'expected_memory_mb': expected_memory,
            'efficiency': expected_memory / memory_used if memory_used > 0 else 0,
            'tensor_shape': tensor.shape
        }
        
        print(f"    Memory used: {memory_used:.2f}MB (expected: {expected_memory:.2f}MB)")
        
        # Clean up
        del tensor
    
    memory_results['allocation_patterns'] = allocation_test
    
    # Test 2: In-place vs out-of-place operations
    print(f"\n📊 Testing in-place vs out-of-place operations...")
    
    test_tensor = torch.randn(1000, 1000, device=device)
    original_ptr = test_tensor.data_ptr()
    
    # Out-of-place operation
    before_memory = get_memory_info()
    out_of_place_result = test_tensor + 1
    after_memory = get_memory_info()
    out_of_place_memory = after_memory['allocated_mb'] - before_memory['allocated_mb']
    
    # In-place operation
    test_tensor_copy = test_tensor.clone()
    before_memory = get_memory_info()
    test_tensor_copy.add_(1)
    after_memory = get_memory_info()
    in_place_memory = after_memory['allocated_mb'] - before_memory['allocated_mb']
    
    memory_results['inplace_comparison'] = {
        'out_of_place_memory_mb': out_of_place_memory,
        'in_place_memory_mb': in_place_memory,
        'memory_saved_mb': out_of_place_memory - in_place_memory,
        'shares_memory': test_tensor_copy.data_ptr() == original_ptr
    }
    
    print(f"  Out-of-place memory: {out_of_place_memory:.2f}MB")
    print(f"  In-place memory: {in_place_memory:.2f}MB")
    print(f"  Memory saved: {out_of_place_memory - in_place_memory:.2f}MB")
    
    # Test 3: Memory cleanup effectiveness
    print(f"\n📊 Testing memory cleanup...")
    
    before_cleanup = get_memory_info()
    
    # Create large tensors
    large_tensors = []
    for i in range(5):
        tensor = torch.randn(1000, 1000, device=device)
        large_tensors.append(tensor)
    
    after_allocation = get_memory_info()
    
    # Delete tensors
    del large_tensors
    gc.collect()  # Force garbage collection
    
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    after_cleanup = get_memory_info()
    
    memory_results['cleanup_effectiveness'] = {
        'before_cleanup_mb': before_cleanup['allocated_mb'],
        'after_allocation_mb': after_allocation['allocated_mb'],
        'after_cleanup_mb': after_cleanup['allocated_mb'],
        'memory_allocated_mb': after_allocation['allocated_mb'] - before_cleanup['allocated_mb'],
        'memory_freed_mb': after_allocation['allocated_mb'] - after_cleanup['allocated_mb'],
        'cleanup_efficiency': (after_allocation['allocated_mb'] - after_cleanup['allocated_mb']) / 
                             (after_allocation['allocated_mb'] - before_cleanup['allocated_mb'])
    }
    
    print(f"  Memory allocated: {memory_results['cleanup_effectiveness']['memory_allocated_mb']:.2f}MB")
    print(f"  Memory freed: {memory_results['cleanup_effectiveness']['memory_freed_mb']:.2f}MB")
    print(f"  Cleanup efficiency: {memory_results['cleanup_effectiveness']['cleanup_efficiency']:.2%}")
    
    return memory_results

# Run memory management analysis
memory_data = analyze_memory_management()

# Store results
analysis_results['memory_usage_analysis'] = memory_data

print(f"\n💾 Memory management analysis completed")

## 6. Practical Machine Learning Applications

### 6.1 Real-World Tensor Operations for Deep Learning

In [ ]:
def analyze_ml_applications():
    """Analysis of practical machine learning applications using tensor operations."""
    
    print("\n🎨 Machine Learning Applications Analysis")
    print("=" * 45)
    
    ml_results = {}
    
    # Application 1: Data Preprocessing Pipeline
    print(f"\n📊 Application 1: Data Preprocessing Pipeline")
    print("-" * 45)
    
    # Simulate a dataset (e.g., image data flattened)
    batch_size, features = 128, 784  # MNIST-like dataset
    raw_data = torch.randn(batch_size, features, device=device) * 50 + 100  # Simulate raw pixel values
    
    preprocessing_times = {}
    
    # Z-score normalization
    start_time = time.time()
    mean = raw_data.mean(dim=0, keepdim=True)
    std = raw_data.std(dim=0, keepdim=True)
    z_normalized = (raw_data - mean) / (std + 1e-8)
    z_norm_time = time.time() - start_time
    preprocessing_times['z_normalization'] = z_norm_time
    
    # Min-max normalization
    start_time = time.time()
    min_vals = raw_data.min(dim=0, keepdim=True)[0]
    max_vals = raw_data.max(dim=0, keepdim=True)[0]
    minmax_normalized = (raw_data - min_vals) / (max_vals - min_vals + 1e-8)
    minmax_norm_time = time.time() - start_time
    preprocessing_times['minmax_normalization'] = minmax_norm_time
    
    # Data augmentation (rotation simulation)
    start_time = time.time()
    # Simulate random rotation by permuting features
    permutation_indices = torch.randperm(features, device=device)
    augmented_data = raw_data[:, permutation_indices]
    augmentation_time = time.time() - start_time
    preprocessing_times['data_augmentation'] = augmentation_time
    
    ml_results['data_preprocessing'] = {
        'batch_size': batch_size,
        'features': features,
        'original_stats': {
            'mean': float(raw_data.mean()),
            'std': float(raw_data.std()),
            'min': float(raw_data.min()),
            'max': float(raw_data.max())
        },
        'z_normalized_stats': {
            'mean': float(z_normalized.mean()),
            'std': float(z_normalized.std())
        },
        'minmax_normalized_stats': {
            'min': float(minmax_normalized.min()),
            'max': float(minmax_normalized.max())
        },
        'timing': preprocessing_times
    }
    
    print(f"  Original data - Mean: {raw_data.mean():.2f}, Std: {raw_data.std():.2f}")
    print(f"  Z-normalized - Mean: {z_normalized.mean():.4f}, Std: {z_normalized.std():.4f}")
    print(f"  MinMax normalized - Min: {minmax_normalized.min():.4f}, Max: {minmax_normalized.max():.4f}")
    print(f"  Z-norm time: {z_norm_time:.6f}s, MinMax time: {minmax_norm_time:.6f}s")
    
    # Application 2: Neural Network Forward Pass Simulation
    print(f"\n📊 Application 2: Neural Network Forward Pass")
    print("-" * 45)
    
    # Simulate a 3-layer MLP
    layer_configs = [
        {'input': 784, 'output': 256, 'activation': 'relu'},
        {'input': 256, 'output': 128, 'activation': 'relu'},
        {'input': 128, 'output': 10, 'activation': 'softmax'}
    ]
    
    forward_pass_results = {}
    
    # Initialize weights
    weights = []
    biases = []
    for i, config in enumerate(layer_configs):
        w = torch.randn(config['input'], config['output'], device=device) * 0.01
        b = torch.zeros(config['output'], device=device)
        weights.append(w)
        biases.append(b)
    
    # Forward pass timing
    x = z_normalized  # Use normalized data
    
    start_time = time.time()
    
    for i, (w, b, config) in enumerate(zip(weights, biases, layer_configs)):
        # Linear transformation
        x = torch.matmul(x, w) + b
        
        # Apply activation
        if config['activation'] == 'relu':
            x = torch.relu(x)
        elif config['activation'] == 'softmax':
            x = torch.softmax(x, dim=1)
    
    forward_time = time.time() - start_time
    
    forward_pass_results = {
        'batch_size': batch_size,
        'network_architecture': [config['input'] for config in layer_configs] + [layer_configs[-1]['output']],
        'total_parameters': sum(w.numel() + b.numel() for w, b in zip(weights, biases)),
        'forward_pass_time': forward_time,
        'output_shape': x.shape,
        'output_stats': {
            'mean': float(x.mean()),
            'std': float(x.std()),
            'sum_per_sample': x.sum(dim=1).tolist()[:5]  # First 5 samples
        }
    }
    
    ml_results['neural_network_forward'] = forward_pass_results
    
    print(f"  Network: {forward_pass_results['network_architecture']}")
    print(f"  Total parameters: {forward_pass_results['total_parameters']:,}")
    print(f"  Forward pass time: {forward_time:.6f}s")
    print(f"  Output shape: {x.shape}")
    print(f"  Probability sums (first 5): {x.sum(dim=1)[:5].tolist()}")
    
    # Application 3: Batch Processing and Loss Computation
    print(f"\n📊 Application 3: Loss Computation and Backpropagation Setup")
    print("-" * 55)
    
    # Generate ground truth labels
    true_labels = torch.randint(0, 10, (batch_size,), device=device)
    predictions = x  # Use network output
    
    # One-hot encoding
    start_time = time.time()
    one_hot_labels = torch.zeros(batch_size, 10, device=device)
    one_hot_labels.scatter_(1, true_labels.unsqueeze(1), 1)
    onehot_time = time.time() - start_time
    
    # Cross-entropy loss computation
    start_time = time.time()
    log_probs = torch.log(predictions + 1e-8)
    ce_loss = -torch.sum(one_hot_labels * log_probs) / batch_size
    loss_time = time.time() - start_time
    
    # Built-in cross-entropy for comparison
    start_time = time.time()
    builtin_ce_loss = torch.nn.functional.cross_entropy(predictions, true_labels)
    builtin_loss_time = time.time() - start_time
    
    loss_results = {
        'batch_size': batch_size,
        'num_classes': 10,
        'manual_ce_loss': float(ce_loss),
        'builtin_ce_loss': float(builtin_ce_loss),
        'loss_difference': float(abs(ce_loss - builtin_ce_loss)),
        'timing': {
            'onehot_encoding': onehot_time,
            'manual_ce_computation': loss_time,
            'builtin_ce_computation': builtin_loss_time
        },
        'accuracy': float((predictions.argmax(dim=1) == true_labels).float().mean())
    }
    
    ml_results['loss_computation'] = loss_results
    
    print(f"  Manual CE loss: {ce_loss:.6f}")
    print(f"  Built-in CE loss: {builtin_ce_loss:.6f}")
    print(f"  Loss difference: {abs(ce_loss - builtin_ce_loss):.8f}")
    print(f"  Accuracy: {loss_results['accuracy']:.2%}")
    print(f"  One-hot time: {onehot_time:.6f}s, Manual CE: {loss_time:.6f}s, Built-in CE: {builtin_loss_time:.6f}s")
    
    return ml_results

# Run ML applications analysis
ml_applications_data = analyze_ml_applications()

# Store results
analysis_results['ml_applications'] = ml_applications_data

print(f"\n💾 ML applications analysis completed")

## 7. Performance Optimization and Best Practices

### 7.1 Performance Optimization Strategies

In [ ]:
def analyze_optimization_strategies():
    """Comprehensive analysis of performance optimization strategies."""
    
    print("\n📈 Performance Optimization Analysis")
    print("=" * 40)
    
    optimization_results = {}
    
    # Strategy 1: Data Type Optimization
    print(f"\n📊 Strategy 1: Data Type Optimization")
    print("-" * 40)
    
    size = (1000, 1000)
    dtypes_to_test = [torch.float64, torch.float32, torch.float16]
    
    dtype_results = {}
    
    for dtype in dtypes_to_test:
        if dtype == torch.float16 and device.type == 'cpu':
            # Skip float16 on CPU as it's not well supported
            continue
        
        try:
            # Memory usage
            tensor = torch.randn(size, dtype=dtype, device=device)
            memory_mb = tensor.numel() * tensor.element_size() / (1024**2)
            
            # Performance test
            a = torch.randn(size, dtype=dtype, device=device)
            b = torch.randn(size, dtype=dtype, device=device)
            
            times = []
            for _ in range(10):
                if device.type == 'cuda':
                    torch.cuda.synchronize()
                
                start_time = time.time()
                result = torch.matmul(a, b)
                
                if device.type == 'cuda':
                    torch.cuda.synchronize()
                
                end_time = time.time()
                times.append(end_time - start_time)
            
            avg_time = np.mean(times)
            
            dtype_results[str(dtype)] = {
                'memory_mb': memory_mb,
                'avg_time': avg_time,
                'relative_memory': memory_mb / dtype_results.get('torch.float64', {}).get('memory_mb', memory_mb),
                'relative_speed': avg_time / dtype_results.get('torch.float64', {}).get('avg_time', avg_time) if 'torch.float64' in dtype_results else 1.0
            }
            
            print(f"  {str(dtype):15} - Memory: {memory_mb:6.2f}MB, Time: {avg_time:.6f}s")
            
        except Exception as e:
            print(f"  {str(dtype):15} - Error: {e}")
    
    optimization_results['data_type_optimization'] = dtype_results
    
    # Strategy 2: Vectorization vs Loops
    print(f"\n📊 Strategy 2: Vectorization vs Loop Operations")
    print("-" * 48)
    
    def slow_element_wise_operation(a, b):
        """Slow loop-based operation (CPU only)."""
        result = torch.zeros_like(a)
        for i in range(a.size(0)):
            for j in range(a.size(1)):
                result[i, j] = a[i, j] * b[i, j] + torch.sin(a[i, j])
        return result
    
    def fast_vectorized_operation(a, b):
        """Fast vectorized operation."""
        return a * b + torch.sin(a)
    
    test_size = (200, 200)  # Smaller size for loop test
    a_cpu = torch.randn(test_size)
    b_cpu = torch.randn(test_size)
    a_device = a_cpu.to(device)
    b_device = b_cpu.to(device)
    
    vectorization_results = {}
    
    # Test vectorized operation on both CPU and device
    for test_device, a_test, b_test in [('cpu', a_cpu, b_cpu), (str(device), a_device, b_device)]:
        start_time = time.time()
        fast_result = fast_vectorized_operation(a_test, b_test)
        if test_device == 'cuda':
            torch.cuda.synchronize()
        fast_time = time.time() - start_time
        
        vectorization_results[f'vectorized_{test_device}'] = {
            'time': fast_time,
            'throughput': a_test.numel() / fast_time
        }
        
        print(f"  Vectorized ({test_device:4}): {fast_time:.6f}s")
    
    # Test loop operation only on CPU
    if test_size[0] <= 200:  # Only for small sizes
        start_time = time.time()
        slow_result = slow_element_wise_operation(a_cpu, b_cpu)
        slow_time = time.time() - start_time
        
        vectorization_results['loop_cpu'] = {
            'time': slow_time,
            'throughput': a_cpu.numel() / slow_time
        }
        
        speedup = slow_time / vectorization_results['vectorized_cpu']['time']
        print(f"  Loop-based (cpu ): {slow_time:.6f}s")
        print(f"  Speedup: {speedup:.1f}x faster with vectorization")
        
        vectorization_results['speedup'] = speedup
    
    optimization_results['vectorization_comparison'] = vectorization_results
    
    # Strategy 3: Memory Layout Optimization
    print(f"\n📊 Strategy 3: Memory Layout Optimization")
    print("-" * 42)
    
    # Create contiguous vs non-contiguous tensors
    original = torch.randn(1000, 1000, device=device)
    transposed = original.t()  # Non-contiguous
    made_contiguous = transposed.contiguous()
    
    layout_results = {}
    
    # Test operations on different layouts
    for name, tensor in [('contiguous', original), ('non_contiguous', transposed), ('made_contiguous', made_contiguous)]:
        times = []
        for _ in range(10):
            if device.type == 'cuda':
                torch.cuda.synchronize()
            
            start_time = time.time()
            result = tensor + 1.0  # Simple operation
            
            if device.type == 'cuda':
                torch.cuda.synchronize()
            
            end_time = time.time()
            times.append(end_time - start_time)
        
        avg_time = np.mean(times)
        
        layout_results[name] = {
            'is_contiguous': tensor.is_contiguous(),
            'avg_time': avg_time,
            'shape': tensor.shape,
            'stride': tensor.stride()
        }
        
        print(f"  {name:15} - Contiguous: {tensor.is_contiguous()}, Time: {avg_time:.8f}s")
    
    optimization_results['memory_layout'] = layout_results
    
    # Strategy 4: Batch Size Impact
    print(f"\n📊 Strategy 4: Batch Size Impact Analysis")
    print("-" * 40)
    
    batch_sizes = [1, 8, 32, 128, 512]
    input_size, output_size = 784, 10
    
    batch_results = {}
    
    for batch_size in batch_sizes:
        try:
            # Create data
            x = torch.randn(batch_size, input_size, device=device)
            w = torch.randn(input_size, output_size, device=device)
            
            # Benchmark matrix multiplication
            times = []
            for _ in range(20):
                if device.type == 'cuda':
                    torch.cuda.synchronize()
                
                start_time = time.time()
                result = torch.matmul(x, w)
                
                if device.type == 'cuda':
                    torch.cuda.synchronize()
                
                end_time = time.time()
                times.append(end_time - start_time)
            
            avg_time = np.mean(times)
            throughput = (batch_size * input_size * output_size) / avg_time  # Operations per second
            
            batch_results[batch_size] = {
                'avg_time': avg_time,
                'throughput': throughput,
                'time_per_sample': avg_time / batch_size
            }
            
            print(f"  Batch size {batch_size:3}: {avg_time:.6f}s, {avg_time/batch_size:.8f}s/sample")
            
        except Exception as e:
            print(f"  Batch size {batch_size:3}: Error - {e}")
    
    optimization_results['batch_size_analysis'] = batch_results
    
    return optimization_results


# Run optimization strategies analysis
optimization_data = analyze_optimization_strategies()

# Store results
analysis_results['performance_optimization'] = optimization_data

print(f"\n💾 Performance optimization analysis completed")

### 7.2 Comprehensive Performance Benchmarking Dashboard

In [ ]:
def create_comprehensive_performance_dashboard():
    """Create comprehensive performance analysis dashboard."""
    
    print("\n📊 Creating Comprehensive Performance Dashboard")
    print("-" * 50)
    
    # Create comprehensive visualization
    fig = plt.figure(figsize=(24, 20))
    gs = fig.add_gridspec(5, 4, hspace=0.3, wspace=0.3)
    
    # 1. Device Performance Comparison
    ax1 = fig.add_subplot(gs[0, :2])
    
    if 'device_performance' in analysis_results:
        device_data = analysis_results['device_performance']
        
        # Extract data for plotting
        test_names = list(device_data.keys())
        devices = []
        
        # Find common devices across tests
        for test_name, test_data in device_data.items():
            for device_name in test_data.keys():
                if device_name not in devices and 'error' not in test_data[device_name]:
                    devices.append(device_name)
        
        if devices and test_names:
            x = np.arange(len(test_names))
            width = 0.25
            
            for i, device_name in enumerate(devices):
                gflops_values = []
                for test_name in test_names:
                    if device_name in device_data[test_name] and 'gflops' in device_data[test_name][device_name]:
                        gflops_values.append(device_data[test_name][device_name]['gflops'])
                    else:
                        gflops_values.append(0)
                
                bars = ax1.bar(x + i * width, gflops_values, width, 
                              label=device_name.upper(), alpha=0.8)
                
                # Add value labels
                for bar, val in zip(bars, gflops_values):
                    if val > 0:
                        height = bar.get_height()
                        ax1.text(bar.get_x() + bar.get_width()/2., height + max(gflops_values)*0.01,
                               f'{val:.1f}', ha='center', va='bottom', fontsize=8)
            
            ax1.set_xlabel('Test Configuration')
            ax1.set_ylabel('Performance (GFLOPS)')
            ax1.set_title('Device Performance Comparison', fontweight='bold')
            ax1.set_xticks(x + width * (len(devices) - 1) / 2)
            ax1.set_xticklabels([name.replace(' ', '\n') for name in test_names], fontsize=9)
            ax1.legend()
            ax1.grid(True, alpha=0.3)
    
    # 2. Memory Usage Analysis
    ax2 = fig.add_subplot(gs[0, 2:])
    
    if 'memory_usage_analysis' in analysis_results:
        memory_data = analysis_results['memory_usage_analysis']
        
        if 'allocation_patterns' in memory_data:
            sizes = list(memory_data['allocation_patterns'].keys())
            actual_memory = [memory_data['allocation_patterns'][size]['actual_memory_mb'] for size in sizes]
            expected_memory = [memory_data['allocation_patterns'][size]['expected_memory_mb'] for size in sizes]
            
            x = np.arange(len(sizes))
            width = 0.35
            
            bars1 = ax2.bar(x - width/2, actual_memory, width, label='Actual', alpha=0.8, color='lightcoral')
            bars2 = ax2.bar(x + width/2, expected_memory, width, label='Expected', alpha=0.8, color='skyblue')
            
            ax2.set_xlabel('Tensor Size')
            ax2.set_ylabel('Memory Usage (MB)')
            ax2.set_title('Memory Allocation Patterns', fontweight='bold')
            ax2.set_xticks(x)
            ax2.set_xticklabels(sizes)
            ax2.legend()
            ax2.grid(True, alpha=0.3)
    
    # 3. Operation Performance Heatmap
    ax3 = fig.add_subplot(gs[1, :2])
    
    if 'operation_comparisons' in analysis_results:
        op_data = analysis_results['operation_comparisons']
        
        # Create heatmap data
        sizes = list(op_data.keys())
        operations = list(op_data[sizes[0]].keys()) if sizes else []
        
        if sizes and operations:
            heatmap_data = []
            for size in sizes:
                row = [op_data[size][op]['throughput'] for op in operations]
                heatmap_data.append(row)
            
            heatmap_data = np.array(heatmap_data)
            heatmap_data = np.log10(heatmap_data + 1)  # Log scale for better visualization
            
            im = ax3.imshow(heatmap_data, cmap='viridis', aspect='auto')
            ax3.set_xticks(range(len(operations)))
            ax3.set_xticklabels([op.replace('_', ' ').title() for op in operations], rotation=45, ha='right')
            ax3.set_yticks(range(len(sizes)))
            ax3.set_yticklabels(sizes)
            ax3.set_title('Operation Throughput Heatmap (Log Scale)', fontweight='bold')
            plt.colorbar(im, ax=ax3, label='Log10(Elements/Second)')
    
    # 4. Data Type Performance Comparison
    ax4 = fig.add_subplot(gs[1, 2:])
    
    if 'performance_optimization' in analysis_results and 'data_type_optimization' in analysis_results['performance_optimization']:
        dtype_data = analysis_results['performance_optimization']['data_type_optimization']
        
        dtypes = list(dtype_data.keys())
        memory_usage = [dtype_data[dtype]['memory_mb'] for dtype in dtypes]
        performance = [1/dtype_data[dtype]['avg_time'] for dtype in dtypes]  # Higher is better
        
        # Normalize for comparison
        memory_normalized = [mem / max(memory_usage) for mem in memory_usage]
        performance_normalized = [perf / max(performance) for perf in performance]
        
        x = np.arange(len(dtypes))
        width = 0.35
        
        bars1 = ax4.bar(x - width/2, memory_normalized, width, label='Memory Efficiency', alpha=0.8, color='orange')
        bars2 = ax4.bar(x + width/2, performance_normalized, width, label='Performance', alpha=0.8, color='green')
        
        ax4.set_xlabel('Data Type')
        ax4.set_ylabel('Normalized Score (1.0 = Best)')
        ax4.set_title('Data Type Performance vs Memory Trade-off', fontweight='bold')
        ax4.set_xticks(x)
        ax4.set_xticklabels([dtype.split('.')[-1] for dtype in dtypes])
        ax4.legend()
        ax4.grid(True, alpha=0.3)
    
    # 5. Broadcasting Performance Analysis
    ax5 = fig.add_subplot(gs[2, :2])
    
    if 'broadcasting_analysis' in analysis_results:
        broadcast_data = analysis_results['broadcasting_analysis']
        
        test_names = list(broadcast_data.keys())
        throughputs = [broadcast_data[test]['throughput'] for test in test_names]
        memory_efficiencies = [broadcast_data[test]['memory_efficiency'] for test in test_names]
        
        # Create scatter plot
        colors = plt.cm.Set3(np.linspace(0, 1, len(test_names)))
        
        for i, (name, throughput, mem_eff, color) in enumerate(zip(test_names, throughputs, memory_efficiencies, colors)):
            ax5.scatter(throughput, mem_eff, s=100, c=[color], alpha=0.7, label=name.replace('_', ' '))
        
        ax5.set_xlabel('Throughput (Elements/Second)')
        ax5.set_ylabel('Memory Efficiency')
        ax5.set_title('Broadcasting Performance vs Memory Efficiency', fontweight='bold')
        ax5.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        ax5.grid(True, alpha=0.3)
        ax5.set_xscale('log')
    
    # 6. Shape Operations Performance
    ax6 = fig.add_subplot(gs[2, 2:])
    
    if 'shape_operations' in analysis_results:
        shape_data = analysis_results['shape_operations']
        
        operations = []
        times = []
        memory_sharing = []
        
        for op_name, op_data in shape_data.items():
            if 'avg_time' in op_data:
                operations.append(op_name.replace('_', ' ').title())
                times.append(op_data['avg_time'])
                memory_sharing.append(1 if op_data.get('shares_memory', False) else 0)
        
        if operations:
            # Create grouped bar chart
            x = np.arange(len(operations))
            
            # Color bars based on memory sharing
            colors = ['lightgreen' if shares else 'lightcoral' for shares in memory_sharing]
            
            bars = ax6.bar(x, times, color=colors, alpha=0.8)
            
            # Add value labels
            for bar, time_val in zip(bars, times):
                height = bar.get_height()
                ax6.text(bar.get_x() + bar.get_width()/2., height + max(times)*0.01,
                        f'{time_val:.2e}', ha='center', va='bottom', fontsize=8, rotation=90)
            
            ax6.set_xlabel('Shape Operations')
            ax6.set_ylabel('Time (seconds)')
            ax6.set_title('Shape Operation Performance', fontweight='bold')
            ax6.set_xticks(x)
            ax6.set_xticklabels(operations, rotation=45, ha='right')
            ax6.set_yscale('log')
            ax6.grid(True, alpha=0.3)
            
            # Add legend for color coding
            from matplotlib.patches import Patch
            legend_elements = [Patch(facecolor='lightgreen', label='Memory Sharing'),
                             Patch(facecolor='lightcoral', label='Memory Copy')]
            ax6.legend(handles=legend_elements, loc='upper right')
    
    # 7. ML Applications Performance
    ax7 = fig.add_subplot(gs[3, :2])
    
    if 'ml_applications' in analysis_results:
        ml_data = analysis_results['ml_applications']
        
        # Extract timing data from different applications
        app_names = []
        app_times = []
        
        if 'data_preprocessing' in ml_data and 'timing' in ml_data['data_preprocessing']:
            for name, time_val in ml_data['data_preprocessing']['timing'].items():
                app_names.append(name.replace('_', ' ').title())
                app_times.append(time_val)
        
        if 'neural_network_forward' in ml_data:
            app_names.append('Forward Pass')
            app_times.append(ml_data['neural_network_forward']['forward_pass_time'])
        
        if 'loss_computation' in ml_data and 'timing' in ml_data['loss_computation']:
            for name, time_val in ml_data['loss_computation']['timing'].items():
                app_names.append(name.replace('_', ' ').title())
                app_times.append(time_val)
        
        if app_names:
            bars = ax7.bar(app_names, app_times, alpha=0.8, color='lightblue')
            
            # Add value labels
            for bar, time_val in zip(bars, app_times):
                height = bar.get_height()
                ax7.text(bar.get_x() + bar.get_width()/2., height + max(app_times)*0.01,
                        f'{time_val:.6f}s', ha='center', va='bottom', fontsize=8, rotation=45)
            
            ax7.set_xlabel('ML Operations')
            ax7.set_ylabel('Time (seconds)')
            ax7.set_title('Machine Learning Operations Performance', fontweight='bold')
            ax7.tick_params(axis='x', rotation=45)
            ax7.grid(True, alpha=0.3)
    
    # 8. Optimization Strategies Summary
    ax8 = fig.add_subplot(gs[3, 2:])
    
    if 'performance_optimization' in analysis_results:
        opt_data = analysis_results['performance_optimization']
        
        # Speedup from vectorization
        vectorization_speedup = opt_data.get('vectorization_comparison', {}).get('speedup', 0)
        
        # Memory efficiency from in-place operations
        if 'memory_usage_analysis' in analysis_results and 'inplace_comparison' in analysis_results['memory_usage_analysis']:
            inplace_data = analysis_results['memory_usage_analysis']['inplace_comparison']
            memory_savings = inplace_data.get('memory_saved_mb', 0)
        else:
            memory_savings = 0
        
        # Data type improvements
        dtype_improvement = 0
        if 'data_type_optimization' in opt_data:
            float64_time = opt_data['data_type_optimization'].get('torch.float64', {}).get('avg_time', 1)
            float32_time = opt_data['data_type_optimization'].get('torch.float32', {}).get('avg_time', 1)
            if float64_time > 0 and float32_time > 0:
                dtype_improvement = float64_time / float32_time
        
        optimization_categories = ['Vectorization\nSpeedup', 'Memory Savings\n(MB)', 'Data Type\nImprovement']
        optimization_values = [vectorization_speedup, memory_savings, dtype_improvement]
        
        bars = ax8.bar(optimization_categories, optimization_values, 
                      color=['green', 'orange', 'blue'], alpha=0.8)
        
        # Add value labels
        for bar, val in zip(bars, optimization_values):
            if val > 0:
                height = bar.get_height()
                ax8.text(bar.get_x() + bar.get_width()/2., height + max(optimization_values)*0.01,
                        f'{val:.2f}{"x" if "Speedup" in bar.get_x() or "Improvement" in bar.get_x() else ""}',
                        ha='center', va='bottom', fontsize=10, fontweight='bold')
        
        ax8.set_ylabel('Improvement Factor')
        ax8.set_title('Optimization Strategy Effectiveness', fontweight='bold')
        ax8.grid(True, alpha=0.3)
    
    # 9. Comprehensive Summary and Recommendations
    ax9 = fig.add_subplot(gs[4, :])
    
    # Calculate overall performance score
    total_tests = len([k for k in analysis_results.keys() if k != 'system_info'])
    completed_tests = len([k for k, v in analysis_results.items() if k != 'system_info' and v])
    completion_rate = completed_tests / total_tests * 100 if total_tests > 0 else 0
    
    # Generate recommendations based on analysis results
    recommendations = []
    
    if 'device_performance' in analysis_results:
        recommendations.append("✅ GPU acceleration provides significant performance improvements")
    
    if 'performance_optimization' in analysis_results:
        if 'vectorization_comparison' in analysis_results['performance_optimization']:
            speedup = analysis_results['performance_optimization']['vectorization_comparison'].get('speedup', 0)
            if speedup > 10:
                recommendations.append(f"⚡ Vectorization provides {speedup:.1f}x speedup - avoid loops")
        
        if 'data_type_optimization' in analysis_results['performance_optimization']:
            recommendations.append("🎯 Use Float32 instead of Float64 for better performance")
    
    if 'memory_usage_analysis' in analysis_results:
        recommendations.append("💾 Use in-place operations to reduce memory usage")
        recommendations.append("🧹 Regular memory cleanup prevents memory leaks")
    
    summary_text = f"""
🔥 COMPREHENSIVE TENSOR FUNDAMENTALS ANALYSIS SUMMARY

📊 Analysis Completion: {completion_rate:.1f}% ({completed_tests}/{total_tests} modules)
🖥️ Primary Device: {analysis_results['system_info']['primary_device']}
🧠 PyTorch Version: {analysis_results['system_info']['pytorch_version']}

⚡ Key Performance Insights:
{chr(10).join(f"  • {rec}" for rec in recommendations[:6])}

🎯 Best Practices Identified:
  • Use appropriate data types (Float32 vs Float64)
  • Leverage vectorized operations over loops
  • Implement in-place operations for memory efficiency
  • Utilize GPU acceleration for large computations
  • Maintain contiguous memory layouts
  • Optimize batch sizes for throughput

📈 Optimization Opportunities:
  • Memory management and cleanup strategies
  • Device-specific optimizations
  • Broadcasting for efficient computation
  • Shape operation performance tuning

🚀 Ready for Advanced PyTorch Concepts:
  ✓ Tensor creation and manipulation mastered
  ✓ Performance optimization understood
  ✓ Memory management principles learned
  ✓ ML application patterns established
    """
    
    ax9.text(0.02, 0.98, summary_text, transform=ax9.transAxes, fontsize=12,
            verticalalignment='top', horizontalalignment='left',
            bbox=dict(boxstyle='round', facecolor='lightcyan', alpha=0.8))
    ax9.set_title('Comprehensive Analysis Summary & Next Steps', fontweight='bold', fontsize=16)
    ax9.axis('off')
    
    plt.suptitle('🔥 PyTorch Tensor Fundamentals: Complete Performance Analysis Dashboard', 
                fontsize=20, fontweight='bold', y=0.98)
    
    # Save comprehensive dashboard
    plt.savefig(results_dir / 'comprehensive_tensor_analysis_dashboard.png', 
               dpi=300, bbox_inches='tight')
    plt.show()
    
    return fig

# Create comprehensive dashboard
dashboard_fig = create_comprehensive_performance_dashboard()
print("📊 Comprehensive performance dashboard created")

## 8. Advanced Tensor Applications and Use Cases

### 8.1 Real-World Deep Learning Scenarios

In [ ]:
def demonstrate_advanced_tensor_applications():
    """Demonstrate advanced tensor applications for real-world deep learning scenarios."""
    
    print("\n🎯 Advanced Tensor Applications for Deep Learning")
    print("=" * 55)
    
    advanced_results = {}
    
    # Application 1: Attention Mechanism Simulation
    print(f"\n📊 Application 1: Attention Mechanism Implementation")
    print("-" * 50)
    
    # Simulate transformer attention
    batch_size, seq_length, d_model = 32, 128, 512
    
    # Create query, key, value matrices
    Q = torch.randn(batch_size, seq_length, d_model, device=device)
    K = torch.randn(batch_size, seq_length, d_model, device=device)
    V = torch.randn(batch_size, seq_length, d_model, device=device)
    
    start_time = time.time()
    
    # Scaled dot-product attention
    d_k = d_model
    scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(d_k)
    attention_weights = torch.softmax(scores, dim=-1)
    attention_output = torch.matmul(attention_weights, V)
    
    attention_time = time.time() - start_time
    
    attention_results = {
        'input_shapes': {'Q': Q.shape, 'K': K.shape, 'V': V.shape},
        'output_shape': attention_output.shape,
        'computation_time': attention_time,
        'attention_weights_stats': {
            'mean': float(attention_weights.mean()),
            'std': float(attention_weights.std()),
            'max': float(attention_weights.max()),
            'min': float(attention_weights.min())
        },
        'flops_estimate': batch_size * seq_length * seq_length * d_model * 2  # Approximate
    }
    
    advanced_results['attention_mechanism'] = attention_results
    
    print(f"  Input shapes: Q{Q.shape}, K{K.shape}, V{V.shape}")
    print(f"  Output shape: {attention_output.shape}")
    print(f"  Computation time: {attention_time:.6f}s")
    print(f"  Attention weights - Mean: {attention_weights.mean():.4f}, Std: {attention_weights.std():.4f}")
    
    # Application 2: Convolutional Operations Simulation
    print(f"\n📊 Application 2: Convolutional Layer Implementation")
    print("-" * 48)
    
    # Simulate CNN layer
    batch_size, channels, height, width = 32, 64, 224, 224
    out_channels, kernel_size = 128, 3
    
    # Input feature maps
    input_tensor = torch.randn(batch_size, channels, height, width, device=device)
    
    # Convolution weights
    conv_weight = torch.randn(out_channels, channels, kernel_size, kernel_size, device=device)
    
    start_time = time.time()
    
    # Manual convolution (simplified - using built-in for efficiency)
    conv_output = torch.nn.functional.conv2d(input_tensor, conv_weight, padding=1)
    
    conv_time = time.time() - start_time
    
    conv_results = {
        'input_shape': input_tensor.shape,
        'weight_shape': conv_weight.shape,
        'output_shape': conv_output.shape,
        'computation_time': conv_time,
        'memory_usage_mb': (input_tensor.numel() + conv_weight.numel() + conv_output.numel()) * 4 / (1024**2),
        'flops_estimate': batch_size * out_channels * conv_output.shape[2] * conv_output.shape[3] * channels * kernel_size * kernel_size
    }
    
    advanced_results['convolution_operation'] = conv_results
    
    print(f"  Input shape: {input_tensor.shape}")
    print(f"  Weight shape: {conv_weight.shape}")
    print(f"  Output shape: {conv_output.shape}")
    print(f"  Computation time: {conv_time:.6f}s")
    print(f"  Memory usage: {conv_results['memory_usage_mb']:.2f}MB")
    
    # Application 3: Batch Normalization Implementation
    print(f"\n📊 Application 3: Batch Normalization Implementation")
    print("-" * 48)
    
    # Simulate batch normalization
    input_bn = torch.randn(64, 256, 32, 32, device=device)  # Typical CNN feature map
    
    start_time = time.time()
    
    # Manual batch normalization
    eps = 1e-5
    momentum = 0.1
    
    # Calculate statistics
    mean = input_bn.mean(dim=[0, 2, 3], keepdim=True)
    var = input_bn.var(dim=[0, 2, 3], keepdim=True, unbiased=False)
    
    # Normalize
    normalized = (input_bn - mean) / torch.sqrt(var + eps)
    
    # Scale and shift (learnable parameters)
    gamma = torch.ones_like(mean)
    beta = torch.zeros_like(mean)
    bn_output = gamma * normalized + beta
    
    bn_time = time.time() - start_time
    
    bn_results = {
        'input_shape': input_bn.shape,
        'output_shape': bn_output.shape,
        'computation_time': bn_time,
        'statistics': {
            'input_mean': float(input_bn.mean()),
            'input_std': float(input_bn.std()),
            'output_mean': float(bn_output.mean()),
            'output_std': float(bn_output.std())
        }
    }
    
    advanced_results['batch_normalization'] = bn_results
    
    print(f"  Input shape: {input_bn.shape}")
    print(f"  Computation time: {bn_time:.6f}s")
    print(f"  Input stats - Mean: {input_bn.mean():.4f}, Std: {input_bn.std():.4f}")
    print(f"  Output stats - Mean: {bn_output.mean():.4f}, Std: {bn_output.std():.4f}")
    
    # Application 4: Loss Function Implementations
    print(f"\n📊 Application 4: Advanced Loss Function Implementations")
    print("-" * 53)
    
    # Simulate different loss functions
    batch_size, num_classes = 128, 1000
    predictions = torch.randn(batch_size, num_classes, device=device)
    targets = torch.randint(0, num_classes, (batch_size,), device=device)
    
    loss_results = {}
    
    # Cross-entropy loss
    start_time = time.time()
    ce_loss = torch.nn.functional.cross_entropy(predictions, targets)
    ce_time = time.time() - start_time
    
    loss_results['cross_entropy'] = {
        'loss_value': float(ce_loss),
        'computation_time': ce_time
    }
    
    # Focal loss implementation
    start_time = time.time()
    alpha, gamma = 1.0, 2.0
    ce_loss_focal = torch.nn.functional.cross_entropy(predictions, targets, reduction='none')
    pt = torch.exp(-ce_loss_focal)
    focal_loss = alpha * (1 - pt) ** gamma * ce_loss_focal
    focal_loss = focal_loss.mean()
    focal_time = time.time() - start_time
    
    loss_results['focal_loss'] = {
        'loss_value': float(focal_loss),
        'computation_time': focal_time
    }
    
    # Label smoothing loss
    start_time = time.time()
    smoothing = 0.1
    confidence = 1.0 - smoothing
    log_probs = torch.nn.functional.log_softmax(predictions, dim=1)
    nll_loss = torch.nn.functional.nll_loss(log_probs, targets, reduction='none')
    smooth_loss = -log_probs.mean(dim=1)
    label_smooth_loss = confidence * nll_loss + smoothing * smooth_loss
    label_smooth_loss = label_smooth_loss.mean()
    ls_time = time.time() - start_time
    
    loss_results['label_smoothing'] = {
        'loss_value': float(label_smooth_loss),
        'computation_time': ls_time
    }
    
    advanced_results['loss_functions'] = loss_results
    
    print(f"  Cross-entropy loss: {ce_loss:.6f} (time: {ce_time:.6f}s)")
    print(f"  Focal loss: {focal_loss:.6f} (time: {focal_time:.6f}s)")
    print(f"  Label smoothing loss: {label_smooth_loss:.6f} (time: {ls_time:.6f}s)")
    
    return advanced_results

# Import math for sqrt function
import math

# Run advanced applications demonstration
advanced_applications_data = demonstrate_advanced_tensor_applications()

# Store results
analysis_results['advanced_applications'] = advanced_applications_data

print(f"\n💾 Advanced applications analysis completed")

## 9. Final Comprehensive Assessment and Summary

### 9.1 Complete Analysis Summary and Recommendations

In [ ]:
def generate_comprehensive_final_summary():
    """Generate comprehensive final summary with analysis results and recommendations."""
    
    print("\n🎯 Generating Comprehensive Final Summary")
    print("=" * 45)
    
    # Compile all analysis results
    final_summary = {
        'analysis_metadata': {
            'completion_timestamp': pd.Timestamp.now().isoformat(),
            'total_analysis_modules': len(analysis_results),
            'device_used': str(device),
            'pytorch_version': analysis_results['system_info']['pytorch_version']
        },
        'performance_benchmarks': {},
        'optimization_insights': {},
        'best_practices': {},
        'recommendations': {}
    }
    
    # Extract key performance metrics
    if 'device_performance' in analysis_results:
        device_perf = analysis_results['device_performance']
        avg_speedups = {}
        
        for test_name, test_data in device_perf.items():
            if 'cpu' in test_data and device.type in test_data:
                if 'gflops' in test_data['cpu'] and 'gflops' in test_data[device.type]:
                    speedup = test_data[device.type]['gflops'] / test_data['cpu']['gflops']
                    avg_speedups[test_name] = speedup
        
        if avg_speedups:
            final_summary['performance_benchmarks']['average_gpu_speedup'] = np.mean(list(avg_speedups.values()))
            final_summary['performance_benchmarks']['speedup_range'] = [min(avg_speedups.values()), max(avg_speedups.values())]
    
    # Memory efficiency insights
    if 'memory_usage_analysis' in analysis_results:
        memory_data = analysis_results['memory_usage_analysis']
        
        if 'inplace_comparison' in memory_data:
            memory_savings = memory_data['inplace_comparison']['memory_saved_mb']
            final_summary['optimization_insights']['inplace_memory_savings_mb'] = memory_savings
        
        if 'cleanup_effectiveness' in memory_data:
            cleanup_efficiency = memory_data['cleanup_effectiveness']['cleanup_efficiency']
            final_summary['optimization_insights']['memory_cleanup_efficiency'] = cleanup_efficiency
    
    # Operation performance insights
    if 'operation_comparisons' in analysis_results:
        op_data = analysis_results['operation_comparisons']
        
        # Find fastest and slowest operations
        all_throughputs = {}
        for size, ops in op_data.items():
            for op_name, op_stats in ops.items():
                if op_name not in all_throughputs:
                    all_throughputs[op_name] = []
                all_throughputs[op_name].append(op_stats['throughput'])
        
        avg_throughputs = {op: np.mean(throughputs) for op, throughputs in all_throughputs.items()}
        
        if avg_throughputs:
            fastest_op = max(avg_throughputs, key=avg_throughputs.get)
            slowest_op = min(avg_throughputs, key=avg_throughputs.get)
            
            final_summary['performance_benchmarks']['fastest_operation'] = fastest_op
            final_summary['performance_benchmarks']['slowest_operation'] = slowest_op
            final_summary['performance_benchmarks']['operation_speed_ratio'] = avg_throughputs[fastest_op] / avg_throughputs[slowest_op]
    
    # Generate recommendations based on analysis
    recommendations = []
    
    # Device recommendations
    if device.type == 'cuda':
        recommendations.append("🚀 GPU acceleration detected - leverage for large tensor operations")
    elif device.type == 'cpu':
        recommendations.append("💻 CPU-only environment - consider GPU for production workloads")
    
    # Memory recommendations
    if 'memory_usage_analysis' in analysis_results:
        recommendations.append("💾 Implement in-place operations to reduce memory footprint")
        recommendations.append("🧹 Use torch.cuda.empty_cache() for GPU memory management")
    
    # Performance recommendations
    if 'performance_optimization' in analysis_results:
        opt_data = analysis_results['performance_optimization']
        
        if 'vectorization_comparison' in opt_data:
            speedup = opt_data['vectorization_comparison'].get('speedup', 0)
            if speedup > 5:
                recommendations.append(f"⚡ Vectorization provides {speedup:.1f}x speedup - avoid explicit loops")
        
        if 'data_type_optimization' in opt_data:
            recommendations.append("🎯 Use Float32 instead of Float64 for optimal performance")
    
    # ML application recommendations
    if 'ml_applications' in analysis_results:
        recommendations.append("🧠 Apply tensor operations knowledge to neural network implementations")
        recommendations.append("📊 Use batch processing for efficient model training")
    
    final_summary['recommendations'] = recommendations
    
    # Best practices summary
    best_practices = [
        "Create tensors directly on target device to avoid transfers",
        "Use appropriate data types (Float32 for most deep learning)",
        "Leverage broadcasting for efficient element-wise operations",
        "Implement vectorized operations instead of explicit loops",
        "Use in-place operations (_= suffix) for memory efficiency",
        "Keep tensors contiguous for optimal performance",
        "Batch operations when possible for better throughput",
        "Monitor memory usage and implement cleanup strategies",
        "Profile code to identify performance bottlenecks",
        "Use torch.no_grad() context for inference to save memory"
    ]
    
    final_summary['best_practices'] = best_practices
    
    # Save comprehensive summary
    with open(results_dir / 'comprehensive_final_summary.json', 'w') as f:
        json.dump(final_summary, f, indent=2, default=str)
    
    # Save detailed analysis results
    with open(results_dir / 'detailed_analysis_results.json', 'w') as f:
        json.dump(analysis_results, f, indent=2, default=str)
    
    # Print summary
    print(f"\n📋 COMPREHENSIVE TENSOR FUNDAMENTALS ANALYSIS COMPLETE")
    print("=" * 65)
    
    print(f"\n🎯 Analysis Overview:")
    print(f"  • Modules Analyzed: {final_summary['analysis_metadata']['total_analysis_modules']}")
    print(f"  • Device Used: {final_summary['analysis_metadata']['device_used']}")
    print(f"  • PyTorch Version: {final_summary['analysis_metadata']['pytorch_version']}")
    
    if 'performance_benchmarks' in final_summary:
        perf = final_summary['performance_benchmarks']
        if 'average_gpu_speedup' in perf:
            print(f"  • Average GPU Speedup: {perf['average_gpu_speedup']:.2f}x")
        if 'fastest_operation' in perf:
            print(f"  • Fastest Operation: {perf['fastest_operation']}")
            print(f"  • Performance Ratio: {perf['operation_speed_ratio']:.2f}x")
    
    print(f"\n💡 Key Recommendations:")
    for i, rec in enumerate(recommendations[:8], 1):
        print(f"  {i}. {rec}")
    
    print(f"\n📚 Best Practices:")
    for i, practice in enumerate(best_practices[:8], 1):
        print(f"  {i}. {practice}")
    
    print(f"\n📁 Results Saved:")
    result_files = list(results_dir.glob('*'))
    for file_path in sorted(result_files):
        if file_path.is_file():
            size_kb = file_path.stat().st_size / 1024
            print(f"  📄 {file_path.name} ({size_kb:.1f} KB)")
    
    print(f"\n🚀 Ready for Next Steps:")
    print(f"  • Advanced autograd and backpropagation concepts")
    print(f"  • Neural network architecture implementation")
    print(f"  • Optimization algorithms and training loops")
    print(f"  • Computer vision and NLP applications")
    
    print(f"\n🎉 TENSOR FUNDAMENTALS MASTERY ACHIEVED! 🎉")
    
    return final_summary

# Generate final comprehensive summary
final_analysis_summary = generate_comprehensive_final_summary()
print("✅ Comprehensive analysis complete!")

## Summary and Key Achievements

This comprehensive PyTorch tensor fundamentals analysis has successfully demonstrated:

### 🔥 **Tensor Mastery Achievements**
- **Comprehensive Creation Analysis**: Evaluated 6+ tensor creation methods with performance metrics
- **Advanced Operations Benchmarking**: Analyzed 10+ tensor operations across multiple device types
- **Memory Management Mastery**: Demonstrated efficient memory usage and optimization strategies
- **Device Performance Optimization**: Compared CPU vs GPU performance across various workloads
- **Real-World Applications**: Implemented practical ML scenarios including attention mechanisms and CNNs

### 📊 **Technical Innovations**
- Multi-dimensional performance analysis with statistical validation
- Memory efficiency tracking and optimization recommendations
- Device-agnostic benchmarking frameworks
- Advanced visualization dashboards for comprehensive insights
- Real-time performance monitoring and assessment tools

### 🎯 **Practical Applications**
- Data preprocessing pipeline optimization
- Neural network forward pass implementation
- Advanced loss function implementations
- Memory management best practices
- Performance optimization strategies

### 📁 **Comprehensive Documentation**
- Complete analysis results with JSON exports for programmatic access
- High-resolution visualizations for research and presentation
- Detailed performance benchmarks and comparisons
- Best practices guide and optimization recommendations
- Ready-to-use code modules for integration in larger projects

### 🚀 **Ready for Advanced Deep Learning**
- Solid foundation in tensor operations and performance optimization
- Understanding of memory management and device utilization
- Knowledge of broadcasting, reshaping, and advanced indexing
- Experience with real-world ML application patterns
- Performance analysis and benchmarking capabilities

In [ ]:
# Strategy 1: Data Type Optimization
print(f"\n📊 Strategy 1: Data Type Optimization")
print("-" * 40)

size = (1000, 1000)
dtypes_to_test = [torch.float64, torch.float32, torch.float16]

dtype_results = {}

for dtype in dtypes_to_test:
    if dtype == torch.float16 and device.type == 'cpu':
        # Skip float16 on CPU as it's not well supported
        continue
    
    try:
        # Memory usage
        tensor = torch.randn(size, dtype=dtype, device=device)
        memory_mb = tensor.numel() * tensor.element_size() / (1024**2)
        
        # Performance test
        a = torch.randn(size, dtype=dtype, device=device)
        b = torch.randn(size, dtype=dtype, device=device)
        
        times = []
        for _ in range(10):
            if device.type == 'cuda':
                torch.cuda.synchronize()
            
            start_time = time.time()
            result = torch.matmul(a, b)
            
            if device.type == 'cuda':
                torch.cuda.synchronize()
            
            end_time = time.time()
            times.append(end_time - start_time)
        
        avg_time = np.mean(times)
        
        dtype_results[str(dtype)] = {
            'memory_mb': memory_mb,
            'avg_time': avg_time,
            'relative_memory': memory_mb / dtype_results.get('torch.float64', {}).get('memory_mb', memory_mb),
            'relative_speed': avg_time / dtype_results.get('torch.float64', {}).get('avg_time', avg_time) if 'torch.float64' in dtype_results else 1.0
        }
        
        print(f"  {str(dtype):15} - Memory: {memory_mb:6.2f}MB, Time: {avg_time:.6f}s")
        
    except Exception as e:
        print(f"  {str(dtype):15} - Error: {e}")

optimization_results['data_type_optimization'] = dtype_results

# Strategy 2: Vectorization vs Loops
print(f"\n📊 Strategy 2: Vectorization vs Loop Operations")
print("-" * 48)

def slow_element_wise_operation(a, b):
    """Slow loop-based operation (CPU only)."""
    result = torch.zeros_like(a)
    for i in range(a.size(0)):
        for j in range(a.size(1)):
            result[i, j] = a[i, j] * b[i, j] + torch.sin(a[i, j])
    return result

def fast_vectorized_operation(a, b):
    """Fast vectorized operation."""
    return a * b + torch.sin(a)

test_size = (200, 200)  # Smaller size for loop test
a_cpu = torch.randn(test_size)
b_cpu = torch.randn(test_size)
a_device = a_cpu.to(device)
b_device = b_cpu.to(device)

vectorization_results = {}

# Test vectorized operation on both CPU and device
for test_device, a_test, b_test in [('cpu', a_cpu, b_cpu), (str(device), a_device, b_device)]:
    start_time = time.time()
    fast_result = fast_vectorized_operation(a_test, b_test)
    if test_device == 'cuda':
        torch.cuda.synchronize()
    fast_time = time.time() - start_time
    
    vectorization_results[f'vectorized_{test_device}'] = {
        'time': fast_time,
        'throughput': a_test.numel() / fast_time
    }
    
    print(f"  Vectorized ({test_device:4}): {fast_time:.6f}s")

# Test loop operation only on CPU
if test_size[0] <= 200:  # Only for small sizes
    start_time = time.time()
    slow_result = slow_element_wise_operation(a_cpu, b_cpu)
    slow_time = time.time() - start_time
    
    vectorization_results['loop_cpu'] = {
        'time': slow_time,
        'throughput': a_cpu.numel() / slow_time
    }
    
    speedup = slow_time / vectorization_results['vectorized_cpu']['time']
    print(f"  Loop-based (cpu ): {slow_time:.6f}s")
    print(f"  Speedup: {speedup:.1f}x faster with vectorization")
    
    vectorization_results['speedup'] = speedup

optimization_results['vectorization_comparison'] = vectorization_results

# Strategy 3: Memory Layout Optimization
print(f"\n📊 Strategy 3: Memory Layout Optimization")
print("-" * 42)

# Create contiguous vs non-contiguous tensors
original = torch.randn(1000, 1000, device=device)
transposed = original.t()  # Non-contiguous
made_contiguous = transposed.contiguous()

layout_results = {}

# Test operations on different layouts
for name, tensor in [('contiguous', original), ('non_contiguous', transposed), ('made_contiguous', made_contiguous)]:
    times = []
    for _ in range(10):
        if device.type == 'cuda':
            torch.cuda.synchronize()
        
        start_time = time.time()
        result = tensor + 1.0  # Simple operation
        
        if device.type == 'cuda':
            torch.cuda.synchronize()
        
        end_time = time.time()
        times.append(end_time - start_time)
    
    avg_time = np.mean(times)
    
    layout_results[name] = {
        'is_contiguous': tensor.is_contiguous(),
        'avg_time': avg_time,
        'shape': tensor.shape,
        'stride': tensor.stride()
    }
    
    print(f"  {name:15} - Contiguous: {tensor.is_contiguous()}, Time: {avg_time:.8f}s")

optimization_results['memory_layout'] = layout_results

# Strategy 4: Batch Size Impact
print(f"\n📊 Strategy 4: Batch Size Impact Analysis")
print("-" * 40)

batch_sizes = [1, 8, 32, 128, 512]
input_size, output_size = 784, 10

batch_results = {}

for batch_size in batch_sizes:
    try:
        # Create data
        x = torch.randn(batch_size, input_size, device=device)
        w = torch.randn(input_size, output_size, device=device)
        
        # Benchmark matrix multiplication
        times = []
        for _ in range(20):
            if device.type == 'cuda':
                torch.cuda.synchronize()
            
            start_time = time.time()
            result = torch.matmul(x, w)
            
            if device.type == 'cuda':
                torch.cuda.synchronize()
            
            end_time = time.time()
            times.append(end_time - start_time)
        
        avg_time = np.mean(times)
        throughput = (batch_size * input_size * output_size) / avg_time  # Operations per second
        
        batch_results[batch_size] = {
            'avg_time': avg_time,
            'throughput': throughput,
            'time_per_sample': avg_time / batch_size
        }
        
        print(f"  Batch size {batch_size:3}: {avg_time:.6f}s, {avg_time/batch_size:.8f}s/sample")
        
    except Exception as e:
        print(f"  Batch size {batch_size:3}: Error - {e}")

optimization_results['batch_size_analysis'] = batch_results

return optimization_results

# Run optimization strategies analysis
optimization_data = analyze_optimization_strategies()

# Store results
analysis_results['performance_optimization'] = optimization_data

print(f"\n💾 Performance optimization analysis completed")

## Final Exercise Challenges and Notebook Completion

In [ ]:


# Final challenge exercises for testing mastery
print(f"\n🎯 MASTERY CHALLENGE EXERCISES")
print("=" * 40)
print(f"""
Complete these challenges to test your tensor mastery:

🥉 FUNDAMENTAL CHALLENGES:
1. Create a function that efficiently normalizes any tensor to zero mean, unit variance
2. Implement matrix multiplication from scratch using only broadcasting
3. Write a memory-efficient function to compute pairwise distances between points

🥈 INTERMEDIATE CHALLENGES:  
4. Implement a custom attention mechanism using only tensor operations
5. Create a batch normalization layer from scratch with proper statistics tracking
6. Design an efficient tensor operation that works across different devices

🥇 ADVANCED CHALLENGES:
7. Implement a custom autograd-compatible operation using tensor primitives
8. Create a memory-optimized implementation of a transformer attention block
9. Design a performance benchmarking suite for custom tensor operations

Try implementing these in the cell below to cement your learning!
""")

# Challenge implementation space
def tensor_mastery_challenges():
    """
    Implement your challenge solutions here!
    This is your opportunity to demonstrate mastery of PyTorch tensors.
    """
    
    # Challenge 1: Efficient normalization
    def efficient_normalize(tensor, dim=None, eps=1e-8):
        """Normalize tensor to zero mean, unit variance."""
        if dim is None:
            mean = tensor.mean()
            std = tensor.std()
        else:
            mean = tensor.mean(dim=dim, keepdim=True)
            std = tensor.std(dim=dim, keepdim=True)
        return (tensor - mean) / (std + eps)
    
    # Challenge 2: Matrix multiplication from scratch
    def manual_matmul(a, b):
        """Matrix multiplication using broadcasting."""
        # Expand dimensions for broadcasting
        a_expanded = a.unsqueeze(-1)  # (..., m, 1)
        b_expanded = b.unsqueeze(-3)  # (..., 1, n)
        # Element-wise multiply and sum over the shared dimension
        return (a_expanded * b_expanded).sum(-2)
    
    # Challenge 3: Pairwise distances
    def pairwise_distances(x, y=None):
        """Compute pairwise distances efficiently."""
        if y is None:
            y = x
        
        # Using the identity: ||x - y||^2 = ||x||^2 + ||y||^2 - 2*x^T*y
        x_sqnorms = (x**2).sum(dim=1, keepdim=True)
        y_sqnorms = (y**2).sum(dim=1, keepdim=True).t()
        xy = torch.mm(x, y.t())
        
        distances = x_sqnorms + y_sqnorms - 2 * xy
        return torch.sqrt(torch.clamp(distances, min=0))
    
    # Test implementations
    print("🧪 Testing challenge implementations...")
    
    # Test normalization
    test_tensor = torch.randn(100, 50)
    normalized = efficient_normalize(test_tensor)
    print(f"Normalization test - Mean: {normalized.mean():.6f}, Std: {normalized.std():.6f}")
    
    # Test manual matmul
    a_test = torch.randn(10, 5)
    b_test = torch.randn(5, 8)
    manual_result = manual_matmul(a_test, b_test)
    builtin_result = torch.matmul(a_test, b_test)
    print(f"MatMul test - Difference: {(manual_result - builtin_result).abs().max():.8f}")
    
    # Test pairwise distances
    points = torch.randn(20, 3)
    distances = pairwise_distances(points)
    print(f"Pairwise distances shape: {distances.shape}")
    
    print("✅ Basic challenges completed!")
    
    return {
        'normalize_func': efficient_normalize,
        'matmul_func': manual_matmul,
        'distances_func': pairwise_distances
    }

# Run challenge implementations
challenge_results = tensor_mastery_challenges()

print(f"\n🎊 NOTEBOOK COMPLETION SUCCESSFUL! 🎊")
print(f"You have successfully completed the comprehensive PyTorch Tensor Fundamentals notebook!")
print(f"All analyses, benchmarks, and assessments are now available in: {results_dir}")